In [ ]:
pip install xgboost==1.2.1

# Import libraries

In [ ]:
cd drive/MyDrive/Colab\ Notebooks/one_lab_cloud

/content/drive/MyDrive/Colab Notebooks/one_lab_cloud


# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
plt.style.use('seaborn')

plotColors = (list(mcolors.TABLEAU_COLORS)+list(mcolors.TABLEAU_COLORS)+list(mcolors.TABLEAU_COLORS))[:23]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
dataset = pd.read_excel('dataset_linear_interpolation.xlsx', sheet_name='weather', header = [0, 1])

# join two rows headers into one, and make it a one row header

In [ ]:
cols = dataset.columns.to_list()
cols = [','.join(y).replace(' ', '')  for y in cols] # remove spaces, and concatenate double headers into single headers
dataset.columns = cols
dataset.columns

Index(['Year(2014),Year', 'Temp.(°C),high', 'Temp.(°C),avg', 'Temp.(°C),low',
       'DewPoint (°C),high', 'DewPoint (°C),avg', 'DewPoint (°C),low',
       'Humidity (%),high', 'Humidity (%),avg', 'Humidity (%),low',
       'SeaLevelPressure(hPa),high', 'SeaLevelPressure(hPa),avg',
       'SeaLevelPressure(hPa),low', 'Visibility (km),high',
       'Visibility (km),avg', 'Visibility (km),low', 'Wind(km/h),high',
       'Wind(km/h),avg', 'GustSpeed(km/h),high', 'Precip.(mm),Precip.(mm)',
       'Events,Events', 'mealybugspecies,Ferrisiavirgata',
       'mealybugspecies,Iceryaseychellarum', 'mealybugspecies,Iceryapurchasi',
       'mealybugspecies,planococcuscitri', 'Predators,Scymnussyriacus',
       'Predators,Cydoniavicina', 'Predators,Chrysoperlacarnea',
       'Predators,Rodalia', 'Parasitoidattackpredators,Homalotylusvicinus',
       'Parasitoidattackpredators,Homalotyloidea',
       'Primaryparasitoids,Leptomastix', 'Primaryparasitoids,Leptomastidae',
       'Primaryparasitoids,Gyr

In [ ]:
weather_columns_names = ['Date','Temp_H','Temp_avg','Temp_L','Dew_H','Dew_avg','Dew_L','Humidity_H',
                         'Humidity_avg','Humidity_L','Pressure_H','Pressure_avg','Pressure_L',
                         'Visibility_H','Visibility_avg','Visibility_L','Wind_H','Wind_avg',
                         'Gust_Speed_H','Precip_sum','Events']

col_indices = list(np.linspace(0, 20, 21, dtype = int))
old_names = dataset.columns[col_indices]
dataset.rename(columns=dict(zip(old_names, weather_columns_names)), inplace=True)

In [ ]:
dataset.columns

Index(['Date', 'Temp_H', 'Temp_avg', 'Temp_L', 'Dew_H', 'Dew_avg', 'Dew_L',
       'Humidity_H', 'Humidity_avg', 'Humidity_L', 'Pressure_H',
       'Pressure_avg', 'Pressure_L', 'Visibility_H', 'Visibility_avg',
       'Visibility_L', 'Wind_H', 'Wind_avg', 'Gust_Speed_H', 'Precip_sum',
       'Events', 'mealybugspecies,Ferrisiavirgata',
       'mealybugspecies,Iceryaseychellarum', 'mealybugspecies,Iceryapurchasi',
       'mealybugspecies,planococcuscitri', 'Predators,Scymnussyriacus',
       'Predators,Cydoniavicina', 'Predators,Chrysoperlacarnea',
       'Predators,Rodalia', 'Parasitoidattackpredators,Homalotylusvicinus',
       'Parasitoidattackpredators,Homalotyloidea',
       'Primaryparasitoids,Leptomastix', 'Primaryparasitoids,Leptomastidae',
       'Primaryparasitoids,Gyranusoideaindica', 'Primaryparasitoids,Aenasius',
       'Hyperparasitoid,Chartocerussubaeneus'],
      dtype='object')

In [ ]:
#dataset.iloc[365, :]
dataset = dataset.drop(365, axis=0) # drop the row that states that year 2015 has begun

dataset = dataset.drop(['Gust_Speed_H'], axis = 1)  # drop column from dataset
dataset = dataset.drop(['Precip_sum'], axis = 1)    # drop column from dataset
dataset = dataset.drop(['Date'], axis = 1)          # drop column from dataset

In [ ]:
dataset.head()

,Temp_H,Temp_avg,Temp_L,Dew_H,Dew_avg,Dew_L,Humidity_H,Humidity_avg,Humidity_L,Pressure_H,Pressure_avg,Pressure_L,Visibility_H,Visibility_avg,Visibility_L,Wind_H,Wind_avg,Events,"mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus"
0,19.0,14.0,11.0,11.0,6.0,0.0,82.0,59.0,22.0,1021.0,1019.0,1018.0,10.0,4.0,1.0,14.0,8.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21.0,16.0,11.0,9.0,6.0,1.0,67.0,53.0,38.0,1024.0,1023.0,1021.0,10.0,6.0,3.0,40.0,13.0,,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20.0,16.0,11.0,12.0,10.0,8.0,88.0,71.0,46.0,1026.0,1024.0,1022.0,10.0,5.0,2.0,19.0,8.0,,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.0,14.0,10.0,13.0,10.0,7.0,94.0,73.0,44.0,1022.0,1019.0,1016.0,10.0,5.0,2.0,16.0,11.0,,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18.0,14.0,10.0,10.0,7.0,3.0,87.0,61.0,34.0,1018.0,1015.0,1013.0,10.0,8.0,3.0,29.0,13.0,,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dataset.iloc[0, 17]

'\xa0 '

#get dummy for categorical events rain, fog, neutral

In [ ]:
dataset['Events']= dataset['Events'].replace('\xa0 ', 'neutral')
dataset['Events']= dataset['Events'].replace('Fog', 'fog')
dataset['Events']= dataset['Events'].replace('Rain', 'rain')
one_hot = pd.get_dummies(dataset['Events'], drop_first=True)
dataset = dataset.drop('Events',axis = 1)
dataset = dataset.join(one_hot)  # joining only neutral and rain, fog was dropped

In [ ]:
dataset.head()

,Temp_H,Temp_avg,Temp_L,Dew_H,Dew_avg,Dew_L,Humidity_H,Humidity_avg,Humidity_L,Pressure_H,Pressure_avg,Pressure_L,Visibility_H,Visibility_avg,Visibility_L,Wind_H,Wind_avg,"mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus",neutral,rain
0,19.0,14.0,11.0,11.0,6.0,0.0,82.0,59.0,22.0,1021.0,1019.0,1018.0,10.0,4.0,1.0,14.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,21.0,16.0,11.0,9.0,6.0,1.0,67.0,53.0,38.0,1024.0,1023.0,1021.0,10.0,6.0,3.0,40.0,13.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,20.0,16.0,11.0,12.0,10.0,8.0,88.0,71.0,46.0,1026.0,1024.0,1022.0,10.0,5.0,2.0,19.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,19.0,14.0,10.0,13.0,10.0,7.0,94.0,73.0,44.0,1022.0,1019.0,1016.0,10.0,5.0,2.0,16.0,11.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,18.0,14.0,10.0,10.0,7.0,3.0,87.0,61.0,34.0,1018.0,1015.0,1013.0,10.0,8.0,3.0,29.0,13.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [ ]:
# rearranging columns so that features come first, then insects come second
cols = dataset.columns.tolist()
cols = cols[:17] + cols[-2:]+cols[17:-2]
# we have 19 features
# and 15 insects
dataset = dataset[cols]
dataset.head()

,Temp_H,Temp_avg,Temp_L,Dew_H,Dew_avg,Dew_L,Humidity_H,Humidity_avg,Humidity_L,Pressure_H,Pressure_avg,Pressure_L,Visibility_H,Visibility_avg,Visibility_L,Wind_H,Wind_avg,neutral,rain,"mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus"
0,19.0,14.0,11.0,11.0,6.0,0.0,82.0,59.0,22.0,1021.0,1019.0,1018.0,10.0,4.0,1.0,14.0,8.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,21.0,16.0,11.0,9.0,6.0,1.0,67.0,53.0,38.0,1024.0,1023.0,1021.0,10.0,6.0,3.0,40.0,13.0,1,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20.0,16.0,11.0,12.0,10.0,8.0,88.0,71.0,46.0,1026.0,1024.0,1022.0,10.0,5.0,2.0,19.0,8.0,1,0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.0,14.0,10.0,13.0,10.0,7.0,94.0,73.0,44.0,1022.0,1019.0,1016.0,10.0,5.0,2.0,16.0,11.0,1,0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18.0,14.0,10.0,10.0,7.0,3.0,87.0,61.0,34.0,1018.0,1015.0,1013.0,10.0,8.0,3.0,29.0,13.0,1,0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dataset.info()
dataset.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 730
Data columns (total 34 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Temp_H                                        730 non-null    float64
 1   Temp_avg                                      730 non-null    float64
 2   Temp_L                                        730 non-null    float64
 3   Dew_H                                         730 non-null    float64
 4   Dew_avg                                       730 non-null    float64
 5   Dew_L                                         730 non-null    float64
 6   Humidity_H                                    730 non-null    float64
 7   Humidity_avg                                  730 non-null    float64
 8   Humidity_L                                    730 non-null    float64
 9   Pressure_H                                    730 non-null    flo

,Temp_H,Temp_avg,Temp_L,Dew_H,Dew_avg,Dew_L,Humidity_H,Humidity_avg,Humidity_L,Pressure_H,Pressure_avg,Pressure_L,Visibility_H,Visibility_avg,Visibility_L,Wind_H,Wind_avg,neutral,rain,"mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus"
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.00000
mean,27.842466,22.664384,17.539726,14.997260,11.254795,6.490411,77.332877,52.124658,22.746575,1016.026027,1013.898630,1011.982192,9.904110,7.308219,4.953425,25.768493,12.057534,0.464384,0.035616,571.487671,19.400000,1.860274,14.680822,23.230137,0.661644,0.475342,0.223288,13.160274,0.120548,2.220548,0.131507,0.565753,0.315068,77.80137
std,6.321180,5.620213,5.191444,5.134674,5.584690,6.166020,13.382016,12.548597,10.748536,4.849802,4.370762,4.518635,0.625573,2.010143,3.119569,16.216297,4.932130,0.499072,0.185459,1027.486492,22.391062,5.584584,45.657048,45.536843,2.730568,1.065776,0.588649,22.278393,0.520356,4.893025,0.394362,2.849047,0.969730,178.27720
min,8.000000,7.000000,2.000000,-1.000000,-6.000000,-16.000000,19.000000,10.000000,4.000000,1006.000000,1005.000000,1001.000000,4.000000,2.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,22.000000,18.000000,13.000000,11.000000,7.000000,3.000000,72.000000,46.000000,15.000000,1013.000000,1010.000000,1008.000000,10.000000,6.000000,2.000000,19.000000,10.000000,0.000000,0.000000,19.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,28.000000,23.000000,18.000000,15.000000,11.000000,7.000000,78.000000,54.000000,23.000000,1016.000000,1014.000000,1012.000000,10.000000,7.000000,4.000000,24.000000,11.000000,0.000000,0.000000,159.000000,10.000000,0.000000,1.000000,5.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.00000
75%,33.000000,28.000000,23.000000,20.000000,16.000000,11.750000,88.000000,59.000000,29.000000,1019.000000,1017.000000,1015.000000,10.000000,9.000000,6.000000,27.000000,14.000000,1.000000,0.000000,487.500000,32.000000,0.000000,8.750000,28.750000,0.000000,0.000000,0.000000,18.000000,0.000000,3.000000,0.000000,0.000000,0.000000,75.75000
max,44.000000,34.000000,30.000000,25.000000,22.000000,18.000000,100.000000,86.000000,67.000000,1049.000000,1029.000000,1027.000000,10.000000,10.000000,10.000000,201.000000,37.000000,1.000000,1.000000,5927.000000,86.000000,40.000000,327.000000,290.000000,22.000000,5.000000,4.000000,121.000000,5.000000,37.000000,2.000000,26.000000,8.000000,1163.00000


In [ ]:
dataset.columns.to_list()

['Temp_H',
 'Temp_avg',
 'Temp_L',
 'Dew_H',
 'Dew_avg',
 'Dew_L',
 'Humidity_H',
 'Humidity_avg',
 'Humidity_L',
 'Pressure_H',
 'Pressure_avg',
 'Pressure_L',
 'Visibility_H',
 'Visibility_avg',
 'Visibility_L',
 'Wind_H',
 'Wind_avg',
 'neutral',
 'rain',
 'mealybugspecies,Ferrisiavirgata',
 'mealybugspecies,Iceryaseychellarum',
 'mealybugspecies,Iceryapurchasi',
 'mealybugspecies,planococcuscitri',
 'Predators,Scymnussyriacus',
 'Predators,Cydoniavicina',
 'Predators,Chrysoperlacarnea',
 'Predators,Rodalia',
 'Parasitoidattackpredators,Homalotylusvicinus',
 'Parasitoidattackpredators,Homalotyloidea',
 'Primaryparasitoids,Leptomastix',
 'Primaryparasitoids,Leptomastidae',
 'Primaryparasitoids,Gyranusoideaindica',
 'Primaryparasitoids,Aenasius',
 'Hyperparasitoid,Chartocerussubaeneus']

In [ ]:
X_cols = ['Temp_H',
          'Temp_avg',
          'Temp_L',
          'Dew_H',
          'Dew_avg',
          'Dew_L',
          'Humidity_H',
          'Humidity_avg',
          'Humidity_L',
          'Pressure_H',
          'Pressure_avg',
          'Pressure_L',
          'Visibility_H',
          'Visibility_avg',
          'Visibility_L',
          'Wind_H',
          'Wind_avg',
          'neutral',
          'rain']

X = dataset[X_cols].values

In [ ]:
y_cols = ['mealybugspecies,Ferrisiavirgata',
          'mealybugspecies,Iceryaseychellarum',
          'mealybugspecies,Iceryapurchasi',
          'mealybugspecies,planococcuscitri',
          'Predators,Scymnussyriacus',
          'Predators,Cydoniavicina',
          'Predators,Chrysoperlacarnea',
          'Predators,Rodalia',
          'Parasitoidattackpredators,Homalotylusvicinus',
          'Parasitoidattackpredators,Homalotyloidea',
          'Primaryparasitoids,Leptomastix',
          'Primaryparasitoids,Leptomastidae',
          'Primaryparasitoids,Gyranusoideaindica',
          'Primaryparasitoids,Aenasius',
          'Hyperparasitoid,Chartocerussubaeneus']

y_tot = dataset[y_cols].values

In [ ]:
from math import sqrt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

# Using minmax scaler

In [ ]:
min_max_scaler1 = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler1.fit_transform(X)
X_df = pd.DataFrame(x_scaled,columns=X_cols)
X_df.head()

,Temp_H,Temp_avg,Temp_L,Dew_H,Dew_avg,Dew_L,Humidity_H,Humidity_avg,Humidity_L,Pressure_H,Pressure_avg,Pressure_L,Visibility_H,Visibility_avg,Visibility_L,Wind_H,Wind_avg,neutral,rain
0,0.305556,0.259259,0.321429,0.461538,0.428571,0.470588,0.777778,0.644737,0.285714,0.348837,0.583333,0.653846,1.0,0.250,0.1,0.020942,0.216216,1.0,0.0
1,0.361111,0.333333,0.321429,0.384615,0.428571,0.500000,0.592593,0.565789,0.539683,0.418605,0.750000,0.769231,1.0,0.500,0.3,0.157068,0.351351,1.0,0.0
2,0.333333,0.333333,0.321429,0.500000,0.571429,0.705882,0.851852,0.802632,0.666667,0.465116,0.791667,0.807692,1.0,0.375,0.2,0.047120,0.216216,1.0,0.0
3,0.305556,0.259259,0.285714,0.538462,0.571429,0.676471,0.925926,0.828947,0.634921,0.372093,0.583333,0.576923,1.0,0.375,0.2,0.031414,0.297297,1.0,0.0
4,0.277778,0.259259,0.285714,0.423077,0.464286,0.558824,0.839506,0.671053,0.476190,0.279070,0.416667,0.461538,1.0,0.750,0.3,0.099476,0.351351,1.0,0.0


In [ ]:
min_max_scaler2 = preprocessing.MinMaxScaler()
y_scaled = min_max_scaler2.fit_transform(y_tot)
y_df = pd.DataFrame(y_scaled,columns=y_cols)
y_df.head()

,"mealybugspecies,Ferrisiavirgata","mealybugspecies,Iceryaseychellarum","mealybugspecies,Iceryapurchasi","mealybugspecies,planococcuscitri","Predators,Scymnussyriacus","Predators,Cydoniavicina","Predators,Chrysoperlacarnea","Predators,Rodalia","Parasitoidattackpredators,Homalotylusvicinus","Parasitoidattackpredators,Homalotyloidea","Primaryparasitoids,Leptomastix","Primaryparasitoids,Leptomastidae","Primaryparasitoids,Gyranusoideaindica","Primaryparasitoids,Aenasius","Hyperparasitoid,Chartocerussubaeneus"
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.002362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Multi-regressor

In [ ]:
objective = "reg:squarederror"
seed = 100
n_estimators = 8
learning_rate = 0.3
gamma = 0.4
subsample = 0.9
colsample_bytree =0.5
reg_alpha = 0.9
reg_lambda = 0.9
silent = False
max_depth = 5
min_child_weight = 0
subsample = 0.1
rmse=[]
rrmse=[]
parameters = {}
parameters['objective'] = objective
parameters['seed'] = seed
parameters['n_estimators'] = n_estimators
parameters['learning_rate'] = learning_rate
parameters['gamma'] = gamma
parameters['colsample_bytree'] = colsample_bytree
parameters['reg_alpha'] = reg_alpha
parameters['reg_lambda'] = reg_lambda
parameters['silent'] = silent

X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=130)

model = xgb.XGBRegressor( objective = objective,
                          seed = seed,
                          n_estimators = n_estimators,
                          learning_rate = learning_rate,
                          gamma = gamma,
                          subsample = subsample,
                          colsample_bytree = colsample_bytree,
                          reg_alpha = reg_alpha,
                          reg_lambda = reg_lambda,
                          silent = silent )

xg_reg_multioutputregressor = MultiOutputRegressor(model)
xg_reg_multioutputregressor.fit(X_train,y_train)
   
preds = xg_reg_multioutputregressor.predict(X_test[:])     
rmse_xgb = mean_squared_error(y_test, preds,squared=False, multioutput='raw_values')
rmse_xgb*100

# Save model to pkl file

In [ ]:
import pickle
filename = 'model.pkl'
pickle.dump(xg_reg_multioutputregressor, open(filename, 'wb'))

# Save scaler to pkl file

In [ ]:
filename = 'scaler1.pkl'
pickle.dump(min_max_scaler1, open(filename, 'wb'))

filename = 'scaler2.pkl'
pickle.dump(min_max_scaler2, open(filename, 'wb'))

# load pkl files

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))
loaded_scaler1 = pickle.load(open('scaler1.pkl', 'rb'))
loaded_scaler2 = pickle.load(open('scaler2.pkl', 'rb'))

# Test pkl files

In [ ]:
test_point = np.array([20.0, 16.0, 11.0, 12.0, 10.0, 8.0, 88.0, 71.0,	46.0,	1026.0,	1024.0,	1022.0,	10.0,	5.0, 2.0, 19.0, 8.0, 1, 0])
scaled_test_point = loaded_scaler1.transform(test_point.reshape(1, -1))
result = loaded_model.predict(scaled_test_point)
result1 = loaded_scaler2.inverse_transform(result)
print(result1)

[[9.8166522e+02 1.1842807e+01 6.0164881e+00 2.3651752e+01 5.5258556e+01
  1.3546879e+00 1.5908954e+00 4.1517127e-01 2.3201799e+01 3.4926429e-01
  3.7693720e+00 3.7759173e-01 4.9333286e+00 6.7338252e-01 2.1085251e+02]]
